In [2]:
import json
import csv
import random
import datetime
import pickle
from gtts import gTTS
import os

# ===================== Card =====================
class Card:
    def __init__(self, question, answer, difficulty='easy'):
        self.question = question
        self.answer = answer
        self.difficulty = difficulty
        self.success_rate = 0.0
        self.attempts = 0
        self.correct = 0

    def update_stats(self, correct):
        self.attempts += 1
        if correct:
            self.correct += 1
        self.success_rate = self.correct / self.attempts

    def to_dict(self):
        return {
            'question': self.question,
            'answer': self.answer,
            'difficulty': self.difficulty,
            'success_rate': self.success_rate,
            'attempts': self.attempts,
            'correct': self.correct
        }

    def generate_audio(self, lang='en'):
        tts = gTTS(text=self.question, lang=lang)
        filename = f"audio_{hash(self.question)}.mp3"
        tts.save(filename)
        return filename

# ===================== Deck =====================
class Deck:
    def __init__(self, name):
        self.name = name
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_hardest_cards(self, n=5):
        return sorted(self.cards, key=lambda c: c.success_rate)[:n]

    def to_dict(self):
        return {
            'name': self.name,
            'cards': [card.to_dict() for card in self.cards]
        }

# ===================== User =====================
class User:
    def __init__(self, name):
        self.name = name
        self.sessions = []

    def add_session(self, session):
        self.sessions.append(session)

    def get_stats(self):
        topic_count = {}
        for session in self.sessions:
            for card in session.deck.cards:
                topic = card.difficulty
                topic_count[topic] = topic_count.get(topic, 0) + 1
        return topic_count

# ===================== Session =====================
class Session:
    def __init__(self, user, deck):
        self.user = user
        self.deck = deck
        self.log = []
        self.timestamp = datetime.datetime.now()

    def run(self):
        for card in self.deck.cards:
            print(f"Q: {card.question}")
            user_answer = input("Your answer: ")
            correct = user_answer.strip().lower() == card.answer.strip().lower()
            card.update_stats(correct)
            self.log.append({
                'question': card.question,
                'your_answer': user_answer,
                'correct': correct,
                'correct_answer': card.answer
            })
            print("Correct!" if correct else f"Wrong! Correct answer: {card.answer}")

    def save_log_csv(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.log[0].keys())
            writer.writeheader()
            writer.writerows(self.log)

    def save_log_json(self, filename):
        with open(filename, 'w') as jsonfile:
            json.dump(self.log, jsonfile, indent=4)

    def save_pickle(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load_pickle(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

# ===================== Example Usage =====================
if __name__ == '__main__':
    user = User("Ali")
    deck = Deck("English Basics")

    # Adding sample cards
    deck.add_card(Card("Hello in Spanish?", "Hola", "easy"))
    deck.add_card(Card("Capital of France?", "Paris", "medium"))
    deck.add_card(Card("2 + 2 * 2 = ?", "6", "easy"))

    session = Session(user, deck)
    session.run()
    user.add_session(session)

    # Save results
    session.save_log_csv("session_results.csv")
    session.save_log_json("session_results.json")
    session.save_pickle("session.pkl")

    # Generate audio for all cards
    for card in deck.cards:
        audio_path = card.generate_audio()
        print(f"Generated audio: {audio_path}")

    # Stats
    print("Hardest Cards:")
    for c in deck.get_hardest_cards():
        print(f"{c.question} - Success Rate: {c.success_rate:.2f}")

    print("Top Learned Topics:")
    print(user.get_stats())


Q: Hello in Spanish?


Wrong! Correct answer: Hola
Q: Capital of France?
Correct!
Q: 2 + 2 * 2 = ?
Correct!
Generated audio: audio_-2844062590306926038.mp3
Generated audio: audio_-1818236259751200457.mp3
Generated audio: audio_-8294575925293029066.mp3
Hardest Cards:
Hello in Spanish? - Success Rate: 0.00
Capital of France? - Success Rate: 1.00
2 + 2 * 2 = ? - Success Rate: 1.00
Top Learned Topics:
{'easy': 2, 'medium': 1}
